In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00


In [8]:
import os
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = ""
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [9]:
class OpenAIAssistant:
  """
  Class to create an OpenAI Assistant object
  """
  def __init__(self, defaults):
    self.defaults = defaults
    self.client = OpenAI(
                  api_key=os.environ.get("OPENAI_API_KEY"),
                  )

  def create_assistant_file(self, file):
    file = client.files.create(file=open(file, "rb"), purpose='assistants')
    return file.id

  def create_external_knowledge(self, knowledge_path):
    included_extensions = ['c', 'cpp', 'csv', 'docx', 'html', 'java' , 'json', 'md', 'pdf', 'php', 'pptx', 'py', 'rb', 'tex', 'txt']

    if os.path.isdir(knowledge_path):
      file_ids = [self.create_assistant_file(os.path.join(knowledge_path, fn)) for fn in os.listdir(knowledge_path)
                    if any(fn.endswith(ext) for ext in included_extensions)]

    elif os.path.isfile(knowledge_path):
      file_ids = [self.create_assistant_file(os.path.join(knowledge_path, fn))]

    else:
      raise ValueError(f"Provided path '{knowledge_path}' is neither a file nor a folder")

    return file_ids

  def create_assistant(self, name, instructions, knowledge_path):
    knowledge_list = self.create_external_knowledge(knowledge_path)
    assistant = client.beta.assistants.create(
                name=name,
                instructions=instructions,
                tools=[{"type": "retrieval"}],
                model= self.defaults["model"],
                file_ids=knowledge_list)
    return assistant


defaults = {
        "model": "gpt-4-turbo-preview",
        "configuration": {
            "temperature": 1,
            "max_tokens": 256,
            "top_p": 1,
            "frequency_penalty": 0,
            "presence_penalty": 0,
        },
    }


In [10]:
name = "Homework assistant"
instructions="You are a homework assistant that helps students with their homework but ensures that it only guides and doesn't do it for them."
knowledge_path = "/content/knowledge_folder" # could be a file/ folder
assistant = OpenAIAssistant(defaults)
new_assistant = assistant.create_assistant(name, instructions, knowledge_path)


In [11]:
new_assistant

Assistant(id='asst_IkGkydGWdVG1gqNdLgHGBPNF', created_at=1710130456, description=None, file_ids=['file-xxxatO1Vp3pCQycrI6GRxlbb', 'file-QPw7qnQ0Eo96bAlHrMEI7E3l'], instructions="You are a homework assistant that helps students with their homework but ensures that it only guides and doesn't do it for them.", metadata={}, model='gpt-4-turbo-preview', name='Homework assistant', object='assistant', tools=[ToolRetrieval(type='retrieval')])

In [12]:
class OpenAIAssistantWrapper:
    def __init__(self, defaults):
        self.defaults = defaults
        self.client = OpenAI(
                  api_key=os.environ.get("OPENAI_API_KEY"),
                  )

    # get assistant
    def get_assistant(self, assistant_id):
      # given assistant_id, retreives assistant obj
      assistant = client.beta.assistants.retrieve(assistant_id)
      return assistant

    # list assistants
    def list_assistant(self):
      # list all assistants available
      assistants = client.beta.assistants.list(order="desc")
      return assistants

    # list assistant files
    def list_assistant_files(self, assistant_id):
      assistant_files = client.beta.assistants.files.list(assistant_id=assistant_id)
      return assistant_files

    # delete assistant
    def delete_assistant(self, assistant_id):
      response = client.beta.assistants.delete(assistant_id)
      return response

In [16]:

openai_wrapper = OpenAIAssistantWrapper(defaults)

# retreive a given assistant
assistant = openai_wrapper.get_assistant('asst_qfXiAczOJuZH2EPCYuMYqcXe')
print('Assistant obj', assistant)

#list assistants
assistants = openai_wrapper.list_assistant()
print('List of assistants', assistants)

# list assistant files
assistant_files = openai_wrapper.list_assistant_files('asst_qfXiAczOJuZH2EPCYuMYqcXe')
print('Assistant files', assistant_files)

#delete assistant
response = openai_wrapper.delete_assistant('asst_qfXiAczOJuZH2EPCYuMYqcXe')
print('Deletion response', response)


Assistant obj Assistant(id='asst_qfXiAczOJuZH2EPCYuMYqcXe', created_at=1709840654, description=None, file_ids=['file-VmsF3EFOPTEt0kKHlyi5K6q6', 'file-v4ynnkKRiWWZ0ic53wa3OUA6'], instructions="You are a homework assistant that helps students with their homework but ensures that it only guides and doesn't do it for them.", metadata={}, model='gpt-4-turbo-preview', name='Homework assistant', object='assistant', tools=[ToolRetrieval(type='retrieval')])
List of assistants SyncCursorPage[Assistant](data=[Assistant(id='asst_IkGkydGWdVG1gqNdLgHGBPNF', created_at=1710130456, description=None, file_ids=['file-QPw7qnQ0Eo96bAlHrMEI7E3l', 'file-xxxatO1Vp3pCQycrI6GRxlbb'], instructions="You are a homework assistant that helps students with their homework but ensures that it only guides and doesn't do it for them.", metadata={}, model='gpt-4-turbo-preview', name='Homework assistant', object='assistant', tools=[ToolRetrieval(type='retrieval')]), Assistant(id='asst_qfXiAczOJuZH2EPCYuMYqcXe', created_at